In [ ]:
# Import libraries
import scipy #for importing .mat data
import pandas as pd #for importing csv/excel data
import numpy as np #for math
import plotly.graph_objects as go #for plotting
import shelve #for importing atlas vis data from db file

In [ ]:
# Import atlas
yba_df = pd.read_excel('Files/YBA_Guide_Spreadsheet.xlsx')
# Display dataframe
yba_df 
# Inititalise atlas variables
atlas_labels = yba_df["Code"].to_list()
atlas_gyrus = yba_df["Gyrus"].to_list()
atlas_region = yba_df["Region"].to_list()
atlas_lobe = yba_df["Lobe"].to_list()
atlas_centroids = yba_df["Centroid"].to_list() 
# Note that the centroids column is a list of strings (which are themselves lists)
# Use eval() on an element of the centroid column to convert the string to a Python list
centroids = []
centroids_x = []
centroids_y = []
centroids_z = []

for string in atlas_centroids:
    centroids.append(eval(string))
    centroids_x.append(-eval(string)[0])
    centroids_y.append(eval(string)[1])
    centroids_z.append(eval(string)[2])

atlas_centroids = centroids
parcel_colors = yba_df["Color"].to_list()


In [ ]:
# Import atlas visualization data (mesh)

atlas_vis_data = shelve.open('Files/atlas_vis_data')
#whole brain
xw = atlas_vis_data['xw'] 
yw = atlas_vis_data['yw'] 
zw = atlas_vis_data['zw'] 
Iw = atlas_vis_data['Iw'] 
Jw = atlas_vis_data['Jw'] 
Kw = atlas_vis_data['Kw'] 
#left hemisphere
xl = atlas_vis_data['xl'] 
yl = atlas_vis_data['yl'] 
zl = atlas_vis_data['zl'] 
Il = atlas_vis_data['Il'] 
Jl = atlas_vis_data['Jl'] 
Kl = atlas_vis_data['Kl'] 
#right hemisphere
xr = atlas_vis_data['xr'] 
yr = atlas_vis_data['yr'] 
zr = atlas_vis_data['zr'] 
Ir = atlas_vis_data['Ir'] 
Jr = atlas_vis_data['Jr'] 
Kr = atlas_vis_data['Kr'] 
#deep left hemisphere
xdl = atlas_vis_data['xdl'] 
ydl = atlas_vis_data['ydl'] 
zdl = atlas_vis_data['zdl'] 
Idl = atlas_vis_data['Idl'] 
Jdl = atlas_vis_data['Jdl'] 
Kdl = atlas_vis_data['Kdl'] 
#deep right hemisphere
xdr = atlas_vis_data['xdr'] 
ydr = atlas_vis_data['ydr'] 
zdr = atlas_vis_data['zdr'] 
Idr = atlas_vis_data['Idr'] 
Jdr = atlas_vis_data['Jdr'] 
Kdr = atlas_vis_data['Kdr'] 
#vertex color for glass brain
trans_parcel_color = atlas_vis_data['trans_parcel_color'] 


atlas_vis_data.close()

Note that this notebook is set up to plot 690 x 690 connectivity matrices in YBA space. Crop corpus callosum parcels from a 696 x 696 matrix prior to use. 

This involves trimming out rows 345:348 and rows 693:696. Same goes for columns.

In [ ]:
# Import connectivity matrix
mat_contents = scipy.io.loadmat('Files/HCP1065_connectivity.mat')
matrix = mat_contents['connectivity']

# Preprocessing matrix
# Add threshold to reduce number of edges
adj_matrix = 1*(matrix>50)
# Create logarithmic matrix
log_matrix = np.log(1+matrix)

In [ ]:
# Set up connected node pairs from adjacency matrix

pairs = []
line_width = []
node_labels = []
node_size = []
node_color = []
line_color = []
for i in range(690):
    if np.sum(adj_matrix[i,:]) > 0: # Check if node has any connections
        node_labels.append(atlas_labels[i]) # Append parcal label if connected
        node_size.append(50*np.sum(adj_matrix,1)[i]) # Node size proportional to number of edges
        node_color.append(np.sum(log_matrix[i,:])) # Node colour corresponds to summed weight of logarithmic connectivity
        for j in range(690):
            if adj_matrix[i,j] != 0: # Check if node i has connection to node j
                pairs.append((i,j)) # Append indices to pairs list if so
                line_color.append(log_matrix[i,j]) # Add edge weight information for line colour
                line_color.append(log_matrix[i,j])
                line_color.append(log_matrix[i,j]) # Repeat twice for reasons
            else:
                continue
    else:
        node_labels.append('') # Blank label for unconnected nodes
        node_size.append(0) # Zero size for unconnected nodes
        node_color.append(0) # No colour for unconnected nodes

# Initialise edges 
x_lines = list()
y_lines = list()
z_lines = list()

# Create the coordinate list for the lines
for p in pairs:
    for i in range(2):
        x_lines.append(centroids_x[p[i]])
        y_lines.append(centroids_y[p[i]])
        z_lines.append(centroids_z[p[i]])
    x_lines.append(None)
    y_lines.append(None)
    z_lines.append(None)


In [ ]:
# Plot network and glass brain

# Add parcel centroids as nodes
fig = go.Figure(data=[go.Scatter3d(x=centroids_z, 
                      y=centroids_y, 
                      z=centroids_x, 
                      mode='markers+text', 
                    marker=dict(sizemode='area',
                                size = node_size,
                               color=node_color, 
                                colorscale='plasma',
                               showscale=True,
                                #cmid=0.5,
                                colorbar_y = 0.95, 
                                colorbar_title='Node Structural Connectivity Strength', colorbar_orientation='h', colorbar_title_side='bottom'
                               ),
                      hoverinfo='text', # Comment out the text sections if labels aren't necessary
                      text=node_labels,
                      textfont = dict(color='black', size=6),
                      showlegend=False)])
# Add edges as lines
fig.add_trace(go.Scatter3d(
                        x=z_lines,
                        y=y_lines,
                        z=x_lines,
                        mode='lines',
                        line = dict(
                        color=line_color,
                        width=2.5, 
                        colorscale='turbo', 
                        colorbar_title='Edge Structural Connectivity Strength', colorbar_orientation='h', colorbar_title_side='top',colorbar_y = 0,
                        showscale=True),
                        hoverinfo='skip',
                        showlegend=False))

# Add whole brain mesh with transparency
fig.add_trace(go.Mesh3d(x=yw, y=zw, z=xw, 
                              i=Iw, j=Jw, k=Kw, 
                              lighting=dict(ambient=0.1,
                                            diffuse=1,
                                            fresnel=3,  
                                            specular=0.5, 
                                            roughness=0.05),
                              lightposition=dict(x=100,
                                                 y=200,
                                                 z=1000),
                           hovertext=None, 
                         vertexcolor=trans_parcel_color,
                             ))
fig.update_layout(title_text="Structural Connectivity Network (YBA)", title_x=0.5, title_y=0.85,
                  font_size=16, font_color="black",
                  width=1000, height=800, autosize=True, 
                  margin=dict(t=2, r=2, b=2, l=2),hovermode='x unified',
                  paper_bgcolor='white',
    scene = dict(xaxis =dict(visible=False, autorange=True), 
                 yaxis =dict(visible=False, autorange=True), 
                 zaxis =dict(visible=False, autorange=True)
                 ,camera_eye=dict(x=0, y=0, z=1.75),
                 aspectmode='data'))
fig.show()


This mini-package has most necessary network visualization functionality, but isn't comprehensive. It's possible to have line width encode connectivity strength, have node colours correspond to lobe/region/gyrus/parcel, have subplots for different hemispheres, and so on. Let me know if those functions are important for your application.

-- Omar Chishti, 23.10.23